### First lab figures

This SQL is to query GL bleeding and Septic shock patients' data for subgroup study

In [1]:
import functools
import numpy as np
import pandas as pd
from scipy.stats import kstest
import matplotlib.pyplot as plt
import pylab as pl
import psycopg2
%matplotlib inline
plt.style.use('ggplot')

In [5]:
# create a database connection
sqluser = 'duhao'
dbname = 'mimic'
schema_name = 'mimiciii'

## Lab ranges

'Normal' ranges for lab values:

Lab Value	|  Lower limit	|  Upper Limit	|  Units
--- | --- | --- | ---
Bicarbonate	| 22	| 32	| mEq/L
BUN	| 6	| 20 | mEq/L
Calcium	| 8.4	| 10.3	| mg/dL
Chloride	| 96	| 108	| mEq/L
Creatinine	| 0.4	| 1.1	| mEq/L
Hemoglobin	| 11.2	 | 15.7	| g/dL
Lactate	| 0.5	| 2	| mmol/L
Magnesium	| 1.6	| 2.6	| mg/dL
Phosphate	| 2.7	| 4.5	| mg/dL
Platelet count	| 150	| 400	| K/uL
Potassium	| 3.3	| 5.1	| mEq/L
Sodium	| 133	| 145	| mEq/L

In [6]:
# Create dictionary of ranges
# Keys should match the lab names in the query below
# Are these values correct given the units? Need to check.

lab_ranges = {'BICARBONATE': [22,32],
              'BUN': [6,20],
              'CALCIUM': [8.4,10.3],
              'CHLORIDE': [96,108],
              'CREATININE': [0.4,1.1],
              'HEMOGLOBIN': [11.2,15.7],
              'LACTATE': [0.5,2.0],
              'MAGNESIUM': [1.6,2.6],
              'PHOSPHATE': [2.7,4.5],
              'PLATELET': [150,400],
              'POTASSIUM': [3.3,5.1],
              'SODIUM': [133,145],
              'FREECALCIUM': [4.64, 5.28]
             }

### SQL: get first laboratory measurements

[Gastrointestinal bleeding](https://en.wikipedia.org/wiki/Gastrointestinal_bleeding): ICD-9 code 578.9  
In MIMIC-III, the description is as follow:   

ICD 9 codes|short description|long description  
--- | --- | --- | ---
    5789| Gastrointest hemorr NOS | Hemorrhage of gastrointestinal tract, unspecified


In [7]:
con = psycopg2.connect(dbname=dbname, user=sqluser, password='duhao123')
cur = con.cursor()
cur.execute('SET search_path to ' + schema_name)

query = \
"""
WITH gi_bleeding_group as
(
    SELECT subject_id, hadm_id,
        CASE
        -- Acute Organ Dysfunction Diagnosis Codes
        WHEN substring(icd9_code,1,4) IN ('5789') THEN 1
        ELSE 0 END AS gi_bleeding
        -- Explicit diagnosis of severe sepsis or septic shock
    FROM diagnoses_icd
), 

infection_group AS
(
    SELECT subject_id, hadm_id,
    CASE
        WHEN substring(icd9_code,1,3) IN ('001','002','003','004','005','008',
               '009','010','011','012','013','014','015','016','017','018',
               '020','021','022','023','024','025','026','027','030','031',
               '032','033','034','035','036','037','038','039','040','041',
               '090','091','092','093','094','095','096','097','098','100',
               '101','102','103','104','110','111','112','114','115','116',
               '117','118','320','322','324','325','420','421','451','461',
               '462','463','464','465','481','482','485','486','494','510',
               '513','540','541','542','566','567','590','597','601','614',
               '615','616','681','682','683','686','730') THEN 1
        WHEN substring(icd9_code,1,4) IN ('5695','5720','5721','5750','5990','7110',
                '7907','9966','9985','9993') THEN 1
        WHEN substring(icd9_code,1,5) IN ('49121','56201','56203','56211','56213',
                '56983') THEN 1
        ELSE 0 END AS infection
    FROM diagnoses_icd
),
-- ICD-9 codes for organ dysfunction - as sourced from Appendix 2 of above paper
organ_diag_group as
(
    SELECT subject_id, hadm_id,
        CASE
        -- Acute Organ Dysfunction Diagnosis Codes
        WHEN substring(icd9_code,1,3) IN ('458','293','570','584') THEN 1
        WHEN substring(icd9_code,1,4) IN ('7855','3483','3481',
                '2874','2875','2869','2866','5734')  THEN 1
        ELSE 0 END AS organ_dysfunction,
        -- Explicit diagnosis of severe sepsis or septic shock
        CASE
        WHEN substring(icd9_code,1,5) IN ('99592','78552')  THEN 1
        ELSE 0 END AS explicit_sepsis
    FROM diagnoses_icd
),
-- Mechanical ventilation
organ_proc_group as
(
    SELECT subject_id, hadm_id,
        CASE
        WHEN substring(icd9_code,1,4) IN ('9670','9671','9672') THEN 1
        ELSE 0 END AS mech_vent
    FROM procedures_icd
),
-- Aggregate above views together
aggregate as
(
    SELECT subject_id, hadm_id,
        CASE
            WHEN hadm_id in
                    (SELECT DISTINCT hadm_id
                    FROM infection_group
                    WHERE infection = 1)
                THEN 1
            ELSE 0 END AS infection,
        CASE
            WHEN hadm_id in
                    (SELECT DISTINCT hadm_id
                    FROM organ_diag_group
                    WHERE explicit_sepsis = 1)
                THEN 1
            ELSE 0 END AS explicit_sepsis,
        CASE
            WHEN hadm_id in
                    (SELECT DISTINCT hadm_id
                    FROM organ_diag_group
                    WHERE organ_dysfunction = 1)
                THEN 1
            ELSE 0 END AS organ_dysfunction,
        CASE
        WHEN hadm_id in
                (SELECT DISTINCT hadm_id
                FROM organ_proc_group
                WHERE mech_vent = 1)
            THEN 1
        ELSE 0 END AS mech_vent
    FROM admissions
),

pvt AS (
  SELECT ie.subject_id, ie.hadm_id, ie.outtime, ie.icustay_id, le.charttime, ad.deathtime, ie.los, gi.gi_bleeding
  , ROUND((cast(ad.admittime as date) - cast(p.dob as date)) / 365.242, 2) as first_admit_age
  , CASE when ad.deathtime between ie.intime and ie.outtime THEN 1 ELSE 0 END AS mort_icu
  , CASE when ad.deathtime between ad.admittime and ad.dischtime THEN 1 ELSE 0 END AS mort_hosp
  -- here we assign labels to ITEMIDs
  -- this also fuses together multiple ITEMIDs containing the same data
  , CASE
        WHEN ag.explicit_sepsis = 1 THEN 1
        WHEN ag.infection = 1 AND ag.organ_dysfunction = 1 THEN 1
        WHEN ag.infection = 1 AND ag.mech_vent = 1 THEN 1
        ELSE 0 END
    AS angus
  , CASE
        when itemid = 50811 then 'HEMOGLOBIN'
        when itemid = 51222 then 'HEMOGLOBIN'
        when itemid = 50813 then 'LACTATE'
        -- Free calcium
        when itemid = 50808 then 'FREECALCIUM'
      ELSE null
      END AS label
  , -- add in some sanity checks on the values
    -- the where clause below requires all valuenum to be > 0,
    -- so these are only upper limit checks
    CASE
      when le.itemid = 50811 and le.valuenum >    50 then null -- g/dL 'HEMOGLOBIN'
      when le.itemid = 51222 and le.valuenum >    50 then null -- g/dL 'HEMOGLOBIN'
      when le.itemid = 50813 and le.valuenum >    50 then null -- mmol/L 'LACTATE'
      -- Free Calcium
      when le.itemid = 50808 and le.valuenum > 500 then null
    ELSE le.valuenum
    END AS valuenum
  FROM icustays ie

  LEFT JOIN labevents le
    ON le.subject_id = ie.subject_id
    AND le.hadm_id = ie.hadm_id
    AND le.charttime between (ie.intime - interval '24' hour)
    AND (ie.intime + interval '24' hour)
    AND le.itemid IN
    (
      -- comment is: LABEL | CATEGORY | FLUID | NUMBER OF ROWS IN LABEVENTS
      51222, -- HEMOGLOBIN | HEMATOLOGY | BLOOD | 752523
      50811, -- HEMOGLOBIN | BLOOD GAS | BLOOD | 89712
      50813, -- LACTATE | BLOOD GAS | BLOOD | 187124
      -- Free calcium
      50808  --FREE CALCIUM | NA | NA | NA
    )
    AND le.valuenum IS NOT null
    AND le.valuenum > 0 -- lab values cannot be 0 and cannot be negative

    LEFT JOIN admissions ad
    ON ie.subject_id = ad.subject_id
    AND ie.hadm_id = ad.hadm_id
    
    LEFT JOIN gi_bleeding_group gi
    ON ie.subject_id = gi.subject_id
    AND ie.hadm_id = gi.hadm_id
    
    LEFT JOIN aggregate ag
    ON ie.subject_id = ag.subject_id
    AND ie.hadm_id = ag.hadm_id
    
    INNER JOIN patients p
    ON ie.subject_id = p.subject_id
    WHERE ROUND((cast(ad.admittime as date) - cast(p.dob as date)) / 365.242, 2) > 15
    -- WHERE ie.subject_id < 10000
),
ranked AS (
SELECT pvt.*, DENSE_RANK() OVER (PARTITION BY
    pvt.subject_id, pvt.hadm_id, pvt.icustay_id, pvt.label ORDER BY cast(pvt.charttime as date)) as drank
FROM pvt
)

SELECT r.subject_id, r.hadm_id, r.icustay_id, r.mort_icu, r.mort_hosp
  -- , max(r.hadm_id) as HADM_ID
  -- , max(r.icustay_id) as ICUSTAY_ID
  , max(r.gi_bleeding) as GI_BLEEDING
  , max(r.angus) as ANGUS
  , max(r.los) as LOS
  , max(r.mort_icu) as MORT_ICU
  , max(r.mort_hosp) as MORT_HOSP
  , max(r.first_admit_age) as FIRST_ADMIT_AGE
  , max(r.charttime) as CHARTTIME
  , max(case when label = 'HEMOGLOBIN' then valuenum else null end) as HEMOGLOBIN_1st
  , max(case when label = 'LACTATE' then valuenum else null end) as LACTATE_1st
  , max(case when label = 'FREECALCIUM' then valuenum else null end) as FREECALCIUM_1st
FROM ranked r
WHERE r.drank = 1
GROUP BY r.subject_id, r.hadm_id, r.icustay_id, r.mort_icu, r.mort_hosp, r.drank
ORDER BY r.subject_id, r.hadm_id, r.icustay_id, r.mort_icu, r.mort_hosp, r.drank;
"""
    
data = pd.read_sql_query(query,con)

In [8]:
print data.shape
print data
#print data[data.gi_bleeding == 1].shape
#print data[data.gi_bleeding == 0].shape

In [24]:
data.to_csv('subgroup_data.csv')


#### From initial to first lab measurements (to be updated)

In [9]:
data_initial = pd.read_csv("subgroup_data.csv", index_col=0)


In [10]:
data_final = data_initial.groupby('subject_id').apply(lambda x: x.sort_values('charttime')).reset_index(drop=True)


In [11]:
print data_final


In [28]:
data_final = data_final.groupby('subject_id').first().reset_index()


In [29]:
#print data_final
print data_final[data_final.gi_bleeding == 1].shape


(1718, 15)


In [30]:
data_final.to_csv('subgroup_data_final.csv')


In [13]:
con = psycopg2.connect(dbname=dbname, user=sqluser, password='duhao123')
cur = con.cursor()
cur.execute('SET search_path to ' + schema_name)

query = \
"""
SELECT * FROM d_icd_diagnoses dd
"""

units = pd.read_sql_query(query,con)
print units

In [6]:
units.to_csv("D_diagnose_ICD.csv")
